In [2]:
from math import pi, atan, sin, cos

import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy import spatial
from tqdm import tqdm

from labvision import images
import filehandlingb
from particletracking import dataframes, statistics

In [23]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [3]:
def get_cgw(df):
    tree = spatial.cKDTree(df[['x', 'y']].values)
    dists, _ = tree.query(tree.data, 2)
    cgw = np.mean(dists[:, 1])
    return cgw

In [4]:
def coarse_order_field(df, cgw, x, y, no_of_neighbours=20):
    """
    Calculate the coarse-grained field characterising local orientation order
    """

    order = df.order.values

    # Generate the lattice nodes to query
    # x, y = np.meshgrid(x, y)
    r = np.dstack((x, y))

    # Get the positions of all the particles
    particles = df[['x', 'y']].values

    # Generate the tree from the particles
    tree = spatial.cKDTree(particles)

    # Query the tree at all the lattice nodes to find the nearest n particles
    # Set n_jobs=-1 to use all cores
    dists, indices = tree.query(r, no_of_neighbours, n_jobs=-1)

    # Calculate all the coarse-grained delta functions (Katira ArXiv eqn 3
    cg_deltas = np.exp(-dists ** 2 / (2 * cgw ** 2)) / (2 * pi * cgw ** 2)

    # Multiply by the orders to get the summands
    summands = cg_deltas * order[indices]

    # Sum along axis 2 to calculate the field
    field = np.sum(summands, axis=2)

    return field

In [28]:
def get_field_threshold(fields, ls, im):
    # Draw a box around an always ordered region of the image to
    # calculate the phi_o
    fields = np.dstack(fields)
    line_selector = LineSelector(im)
    op1, op2 = line_selector.points
    phi_o = np.mean(
        fields[op1[1] // ls:op2[1] // ls, op1[0] // ls:op2[0] // ls, :])

    # Repeat for disordered
    line_selector = LineSelector(im)
    dp1, dp2 = line_selector.points
    phi_d = np.mean(
        fields[dp1[1] // ls:dp2[1] // ls, dp1[0] // ls:dp2[0] // ls, :])

    field_threshold = (phi_o + phi_d) / 2
    return field_threshold


class LineSelector:
    def __init__(self, im):
        cv2.namedWindow('line', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('line', 960, 540)
        cv2.setMouseCallback('line', self.record)
        self.points = []
        while True:
            cv2.imshow('line', im)
            key = cv2.waitKey(1) & 0xFF
            if len(self.points) == 2:
                break
        cv2.destroyAllWindows()

    def record(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.points.append([x, y])

In [5]:
direc = "/media/data/Data/FirstOrder/Interfaces/RecordFluctuatingInterfaceJanuary2020/Quick/first_frames"
savename = f"{direc}/data_new.hdf5"

files = filehandling.get_directory_filenames(direc+'/*.png')
ims = [images.load(f, 0) for f in tqdm(files, 'Loading images')]
ims = [images.bgr_to_gray(im) for im in ims]
circles = [images.find_circles(im, 27, 200, 7, 16, 16)
           for im in tqdm(ims, 'Finding Circles')]

data = dataframes.DataStore(savename, load=False)
for f, info in tqdm(enumerate(circles), 'Adding Circles'):
    data.add_tracking_data(f, info, ['x', 'y', 'r'])

calc = statistics.PropertyCalculator(data)
calc.order()

Finding Circles: 100%|██████████| 50/50 [00:03<00:00, 14.56it/s]
Adding Circles: 50it [00:00, 883.55it/s]


[########################################] | 100% Completed |  0.3s


In [6]:
average_sep = get_cgw(data.df.loc[0])
average_sep

32.981296912510686

In [54]:
lattice_spacing = 10
x = np.arange(0, ims[0].shape[1], lattice_spacing)
y = np.arange(0, ims[0].shape[0], lattice_spacing)
x, y = np.meshgrid(x, y)

In [55]:
consts = np.array([0.2, 0.4, 0.5, 0.6, 0.8, 1, 1.5, 2, 2.5, 3, 4, 5])
consts = np.array([1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4])
cgws = consts * average_sep

In [56]:
fields = [coarse_order_field(data.df.loc[0], cgw, x, y) for cgw in cgws]

In [33]:
def imshow_grid(fs):
    fig, ax = plt.subplots(3, 4)
    for i, f in enumerate(fs):
        ax[i//4, i%4].imshow(f)
        ax[i//4, i%4].set_title(consts[i])
        ax[i//4, i%4].set_axis_off()
        ax[i//4, i%4]
    plt.tight_layout()


In [34]:
imshow_grid(fields)

In [57]:
field_threshold = get_field_threshold(fields, lattice_spacing, ims[0])

In [58]:
thresholded_fields = [field < field_threshold for field in fields]

In [80]:
imshow_grid(thresholded_fields)

In [44]:
def find_contours(f, t):
    t_low = t - 0.02 * t
    t_high = t + 0.02 * 5
    new_f = (f < t_high) * (f > t_low)
    new_f = np.uint8(new_f)
    contours = images.find_contours(new_f)
    contours = images.sort_contours(contours)
    try:
        return contours[-1]
    except IndexError as e:
        print("Only one contour")
        return contours

In [60]:
contours = [find_contours(f, field_threshold) for f in fields]

Only one contour
Only one contour


In [63]:
annotated_ims = [images.draw_contours(images.gray_to_bgr(im), c*lattice_spacing) for im, c in zip(ims, contours)]

In [64]:
imshow_grid(annotated_ims)

In [50]:
ims[0].shape

(1853, 2102)

In [52]:
fields[0].shape

(184, 210)

In [74]:
c = contours[6].copy()

In [75]:
c *= 10

In [76]:
c.shape

(267, 1, 2)

In [81]:
plt.plot(c[:, 0, 0], c[:, 0, 1], 'r')
plt.imshow(ims[0])

In [82]:
def get_angle(im):
    ls = LineSelector(im)
    p1, p2 = ls.points
    m = (p2[1] - p1[1]) / (p2[0] - p1[0])
    a = -atan(m)
    c = np.array([i // 2 for i in np.shape(im)])[::-1]
    return a, c, p1, p2

In [84]:
a, c, p1, p2 = get_angle(ims[0])

In [90]:
a

0.49670896462094993

In [242]:
dx = p2[0] - p1[0]
dy = p2[1] - p1[1]
p0 = (p1[0]-dx, p1[1]-dy)
p3 = (p2[0]+dx, p2[1]+dy)
line = LineString((p0, p3))

In [259]:
N = 100
line_x = np.linspace(p1[0], p2[0], N)
line_y = np.linspace(p1[1], p2[1], N)

In [244]:
from shapely.geometry import LineString, LinearRing

In [249]:
contour = contours[6].copy().squeeze()*10
# contour.add(contour[0, :])
contour = np.vstack((contour, contour[0, :]))
contour_line = LineString(contour)

In [250]:
contour.shape

(268, 2)

In [205]:
from shapely import affinity

In [209]:
from shapely.geometry.multipoint import MultiPoint
from shapely.geometry import Point

In [266]:
crossings = []
dists = []
for (xp, yp) in zip(line_x, line_y):
    point = Point(xp, yp)
    line_rot = affinity.rotate(line, 90, point)
    crosses = contour_line.intersection(line_rot)
    if crosses.geom_type == 'Point':
        dist = crosses.distance(point)
        cross = crosses.x, crosses.y
#         print(cross)
    elif crosses.geom_type == 'MultiPoint':
        distances = [p.distance(point) for p in crosses]
        cross = crosses[np.argmin(distances)]
        cross = cross.x, cross.y
        dist = np.min(distances)
    else:
        cross = xp, yp
        dist = 0
    crossings.append(cross)
    dists.append(dist)
crossings = np.array(crossings)

In [267]:
plt.plot(crossings[:, 0], crossings[:, 1])
cp = contour_line.coords.xy
plt.plot(cp[0], cp[1], '--')
plt.plot(line.coords.xy[0], line.coords.xy[1])

In [269]:
plt.plot(dists)

In [271]:
sp = np.fft.fft(dists)
N = len(dists)
freq = np.fft.fftfreq(N, L/N)

In [275]:
xplot = freq[1: N//2]
yplot = L * np.abs(sp[1: N// 2]) ** 2
plt.loglog(xplot, yplot, '.')